In [99]:
import pandas as pd
Data=pd.read_excel("../../더했다간미워할거야.xlsx")
data=pd.DataFrame(Data)

In [100]:
data=data[["(나)가족의 수입 만족도","(라)가족관계 만족도","가족생활에 대한 만족도","혼인상태"]]

### 파생변수

In [113]:
## 0: 미혼
## 1: 기혼&배우자(x)
## 2: 기혼&배우자(o)
혼인dc={0:0,5:0,2:1,3:1,1:2,4:2}

In [114]:
data["혼인상태"]=data["혼인상태"].apply(lambda x: 혼인dc[x])

In [115]:
data.corr(method="spearman").style.background_gradient(cmap="PuBu")

,(나)가족의 수입 만족도,(라)가족관계 만족도,가족생활에 대한 만족도,혼인상태
(나)가족의 수입 만족도,1.000000,0.366313,0.331271,0.001750
(라)가족관계 만족도,0.366313,1.000000,0.636134,0.036232
가족생활에 대한 만족도,0.331271,0.636134,1.000000,0.071705
혼인상태,0.001750,0.036232,0.071705,1.000000


> 부가적인 설명/

# 독립성 검정

In [56]:
import scipy.stats as sps
import itertools as it

In [57]:
# 묶음 만들기
pairs=it.combinations(data.columns,r=2)

### 기대도수 확인...

In [58]:
cols=[list(col) for col in pairs]

In [64]:
data1=pd.crosstab(data[cols[0][0]],data[cols[0][1]])

In [88]:
data1.columns.name

'(라)가족관계 만족도'

## 결과해석
|위치|설명|
|--|--|
|sps.chi2_contingency[1]|pvalue|
|sps.chi2_contingency[2]|dof|
|sps.chi2_contingency[3]|기대도수|

### 결과 해석 방법
(1) 자유도가
1
인경우 전체 사례수가
30
보다 크면서 각 셀의 빈도가
5
이상일 때 적용가능.         
(2) 사례수가
30
보다 크면서
5
미만의 기대빈도(Expected frequency)의 셀이 전체 칸의
20%
보
다 적고,   모든 셀에
1.0
이상의 기대빈도가 있다면 척도에 관계없이
2
카이제곱
검정법을 사용.          
(3) 각 셀의 기대빈도가
5
미만인 경우 변수들의 범주를 묶고 다시 진행

In [82]:
df=pd.DataFrame(sps.chi2_contingency(data1,correction=False)[3])

In [90]:
df.index.name=data1.index.name

In [93]:
df.columns.name=data1.columns.name

In [94]:
df

(라)가족관계 만족도,0,1,2,3,4
(나)가족의 수입 만족도,,,,,
0,1.323612,6.022434,26.108245,74.420079,10.125631
1,9.826136,44.708918,193.820527,552.474481,75.169938
2,16.455412,74.872126,324.583006,925.205552,125.883904
3,11.800337,53.691531,232.761638,663.473920,90.272574
4,0.594504,2.704992,11.726584,33.425967,4.547953


In [97]:
df[df>5]

(라)가족관계 만족도,0,1,2,3,4
(나)가족의 수입 만족도,,,,,
0,NaN,6.022434,26.108245,74.420079,10.125631
1,9.826136,44.708918,193.820527,552.474481,75.169938
2,16.455412,74.872126,324.583006,925.205552,125.883904
3,11.800337,53.691531,232.761638,663.473920,90.272574
4,NaN,NaN,11.726584,33.425967,NaN


### 모듈만들기

In [117]:
import scipy.stats as sps
import itertools as it
def chi2_result(data):
    #모든 행들의 pair 만들기
    cols=[list(col) for col in it.combinations(data.columns,r=2)]
    result={"Pass":[],"Fail":[]}
    #전역 탐색
    for col in cols:
        #crosstab 만들기
        step1=pd.crosstab(data[col[0]],data[col[1]])
        #chi2 검사
        k2=sps.chi2_contingency(step1,correction=False)
        df=pd.DataFrame(k2[3])
        df.index.name=step1.index.name
        df.columns.name=step1.columns.name
        step3={"Pairs":col,'DataFrame':df,"pvalue":k2[1]}
        if df[df>5].isna().sum().sum():
            result["Fail"].append(step3)
        else:
            result['Pass'].append(step3)
    return result        

In [120]:
chi2_result(data)["Pass"]

[{'Pairs': ['(나)가족의 수입 만족도', '혼인상태'],
  'DataFrame': 혼인상태                    0           1           2
  (나)가족의 수입 만족도                                    
  0               20.615255   79.813797   17.570948
  1              153.042064  592.515984  130.441952
  2              256.293045  992.261357  218.445597
  3              183.790241  711.560292  156.649467
  4                9.259394   35.848570    7.892036,
  'pvalue': 1.7910790396153495e-13},
 {'Pairs': ['(라)가족관계 만족도', '혼인상태'],
  'DataFrame': 혼인상태                  0            1           2
  (라)가족관계 만족도                                     
  0              6.988222    27.055524    5.956254
  1             31.796411   123.102636   27.100953
  2            137.842681   533.670219  117.487100
  3            392.912787  1521.196859  334.890353
  4             53.459899   206.974762   45.565339,
  'pvalue': 4.504082839176538e-18},
 {'Pairs': ['가족생활에 대한 만족도', '혼인상태'],
  'DataFrame': 혼인상태                   0            1           2
  

In [118]:
chi2_result(data)["Fail"][0]["DataFrame"]

(라)가족관계 만족도,0,1,2,3,4
(나)가족의 수입 만족도,,,,,
0,1.323612,6.022434,26.108245,74.420079,10.125631
1,9.826136,44.708918,193.820527,552.474481,75.169938
2,16.455412,74.872126,324.583006,925.205552,125.883904
3,11.800337,53.691531,232.761638,663.473920,90.272574
4,0.594504,2.704992,11.726584,33.425967,4.547953


In [111]:
chi2_result(data)["Fail"][2]["DataFrame"]

혼인상태,0,1,2,3,4
(나)가족의 수입 만족도,,,,,
0,15.552440,60.720695,19.093102,2.018508,20.615255
1,115.457095,450.773976,141.742008,14.984857,153.042064
2,193.351094,754.892036,237.369321,25.094504,256.293045
3,138.653954,541.340437,170.219854,17.995513,183.790241
4,6.985418,27.272855,8.575715,0.906618,9.259394


In [112]:
chi2_result(data)["Fail"][3]["DataFrame"]

가족생활에 대한 만족도,0,1,2,3,4,5,6,7
(라)가족관계 만족도,,,,,,,,
0,2.602356,0.392597,0.897364,1.076837,7.279865,5.272013,19.697140,2.781828
1,11.840718,1.786315,4.083006,4.899607,33.123388,23.987661,89.621985,12.657319
2,51.331464,7.743971,17.700505,21.240606,143.595345,103.990466,388.526080,54.871565
3,146.317443,22.073752,50.454291,60.545149,409.310432,296.418957,1107.471677,156.408301
4,19.908020,3.003365,6.864835,8.237801,55.690970,40.330903,150.683118,21.280987
